# Diffuse and Specular lighting with Texture
Toggles:
- l: to toggle between point and directional light source (Default: Directional light)
- p: to toggle between pause and rotation of the light source (Default: Pause)

Press ESC to exit.

In [1591]:
# Project 1 Remi Roper

In [1592]:
import glm
from objloader import Obj
import numpy as np
import pygame
import moderngl
from math import cos, sin, sqrt
#from LoadObject import getObjectData
import ctypes
#ctypes.windll.user32.SetProcessDPIAware()

In [1593]:
class SceneBound:
    def __init__(self, coords):
        self.boundingBox = [
        np.min(coords,0),
        np.max(coords,0)
        ]
        self.center = (self.boundingBox[0] + self.boundingBox[1])/2
        dVector = (self.boundingBox[1] - self.boundingBox[0])
        self.radius = sqrt(dVector[0]*dVector[0] + dVector[1]*dVector[1]+dVector[2]*dVector[2])/2
    def __str__(self):
        return f"boundingBox:{self.boundingBox}, enter: {self.center}, Radius:{self.radius}."

def get_triangle_normal(vertexList):
    e1 = glm.vec3(vertexList[1]) - glm.vec3(vertexList[0])
    e2 = glm.vec3(vertexList[2]) - glm.vec3(vertexList[0])
    return list(glm.normalize(glm.cross(e1, e2)))

def compute_triangle_normal_coords(position_coord):
    i = 0
    normals = []
    while i < len(position_coord):
        normal = get_triangle_normal(position_coord[i:i+3])
        normals.append(normal)
        normals.append(normal)
        normals.append(normal)
        i += 3
    return np.array(normals)

def getObjectData(filePath, normal=False, texture = False):
    geometry = Obj.open(filePath)
    position_coord = np.array([geometry.vert[f[0]-1] for f in geometry.face])
    if normal==True:
        if geometry.norm:
            normal_coord = np.array([geometry.norm[f[2]-1] for f in geometry.face])
            print("Normal exists")
        else:
            normal_coord = compute_triangle_normal_coords(position_coord)
            print("Normal computed.")
    if texture==True:
        if geometry.text:
            texture_coord = np.array([[geometry.text[f[1]-1][0],geometry.text[f[1]-1][1]] for f in geometry.face])
            print ("texture exists")
        else:
            texture_coord = np.array([[0.5,0.5] for f in geometry.face])
            print("No texture")
    if (normal==False and texture == False):
        vertex_data = position_coord.astype("float32").flatten()
    elif texture == False:
        vertex_data = np.concatenate((position_coord,normal_coord),axis=1).astype("float32").flatten()
    elif normal == False:
        vertex_data = np.concatenate((position_coord,texture_coord),axis=1).astype("float32").flatten()
    else:
        vertex_data = np.concatenate((position_coord,normal_coord,texture_coord),axis=1).astype("float32").flatten()
    return [vertex_data, SceneBound(position_coord)]



### Read data from an OBJ modelfile

In [1594]:
# Data from https://github.com/thinks/platonic-solids/blob/master/models/
files = ["teapot_with_texCoords.obj", "cube.obj", "20_icosahedron.obj"] # Teapot, floor, and light source

In [1595]:
teapot_object = getObjectData(files[0], normal=True, texture=True)
teapot_shadow = getObjectData(files[0], normal=False, texture=False)
box_object = getObjectData(files[1], normal=True, texture=True)
light_object = getObjectData(files[2], normal=True)

Normal exists
texture exists
Normal exists
texture exists
Normal computed.


In [1596]:
box_object_transformation = glm.scale(glm.vec3(10,0.05,10))*glm.translate(glm.vec3(0,-1,0))
teapot_object_transformation = glm.scale(glm.vec3(0.3))*glm.translate(glm.vec3(0,7.875,0))
light_object_scaletransformation = glm.scale(glm.vec3(0.1))

### Initialize pygame and create a window with OpenGL context.

In [1597]:
pygame.init() # Initlizes its different modules. Display module is one of them.
clock = pygame.time.Clock()
window = pygame.display.set_mode((1000, 800), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE , vsync=True) 
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION,4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION,1)
pygame.display.set_caption(title = "Project 1: Remi Roper")
gl = moderngl.get_context()  # Get Previously created context.
gl.enable(gl.DEPTH_TEST)
width, height = window.get_size()
aspect_ratio = width/height

/tmp/ipykernel_211978/3492503202.py:3: Warning: regular vsync for OpenGL not available
  window = pygame.display.set_mode((1000, 800), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE , vsync=True)


#### Load Texture images, Create texture and sampler objects

In [1598]:
def load_image(image_file, channels="RGBA",flip_x=False, flip_y=False):
    # pygame.image.load() will return the 
    # object that has image 
    texture_img = pygame.image.load(image_file) 
    if (flip_x or flip_y):
        texture_img=pygame.transform.flip(texture_img, flip_x, flip_y)
    texture_data = pygame.image.tobytes(texture_img,channels,True)
    #w, h = texture_img.get_size()
    return texture_data, texture_img.get_size()

In [1599]:
img_data, img_size  = load_image("gold.jpg", "RGB")
_texture = gl.texture(img_size, 3, img_data)
_texture.build_mipmaps()
specular_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

img_data, img_size  = load_image('grass.jpg', "RGB")
_texture = gl.texture(img_size, 3, img_data)
_texture.build_mipmaps()
diffuse_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

### Push the Geometry Data to the GPU buffer.

In [1600]:
teapot_vertex_buffer = gl.buffer(teapot_object[0])
Shadow_vertex_buffer = gl.buffer(teapot_shadow[0])
box_object_buffer = gl.buffer(box_object[0])
light_object_buffer = gl.buffer(light_object[0])

### Write shader code, Create shader program(s) and create renderables by connecting the buffers to shader program

In [1601]:
def queryProgramParameters(program):
    for name in program:
        member = program[name]
        print(name, type(member), member)

#### Shared Vertex Shader code

In [1602]:
shared_vertex_shader_code = '''#version 330 core

layout (location = 0) in vec4 in_position;
layout (location = 1) in vec3 in_normal;
layout (location = 2) in vec2 in_uv;

uniform mat4 model;
uniform mat4 view, perspective;

out vec2 f_uv;
out vec3 f_normal;
out vec3 f_position;

void main() {
    f_position = (model*in_position).xyz;
    f_normal = normalize(mat3(transpose(inverse(model)))*in_normal);
    f_uv = in_uv;
    gl_Position = perspective*view*vec4(f_position,1.);
}
'''

shadow_vertex_shader_code = '''#version 330 core

layout (location = 0) in vec3 in_position;

uniform mat4 model;
uniform mat4 view, perspective;

uniform vec3 Q, n; // Shadow Receiver plane information
uniform vec4 L; //Point light

const float bias = 0.001;

void main() {
    vec3 wPosition = (model* vec4(in_position,1)).xyz;
    vec3 projector_direction = L.xyz; // directional
    if (L.w > 0) projector_direction = wPosition - L.xyz; // Point light
    
    float t = dot((Q - wPosition ),n) / dot((projector_direction),n);
    vec3 shadowPosition = wPosition + t *projector_direction + bias*n;
    gl_Position = perspective* view * vec4(shadowPosition,1);
}
'''

#### Main Progam and renderables for Floor and Teapot Object

In [1603]:
print("Main Program Parameters")
mainShaderProgram = gl.program(
  vertex_shader = shared_vertex_shader_code,
  fragment_shader = '''#version 330 core

in vec3 f_position;
in vec3 f_normal;
in vec2 f_uv;

uniform vec3 eye;
uniform sampler2D map;
uniform vec4 light;

uniform bool metal;
const float shininess = 25.0;

layout (location = 0) out vec4 out_color;

void main() {
    vec3 N = normalize(f_normal);
    vec3 L = normalize(light.xyz);
    if (light.w > 0.) L = normalize(light.xyz-f_position);
    
    vec3 V = normalize(eye-f_position);
    vec3 H = normalize(L+V);
    
    vec3 baseColor = texture(map, f_uv).rgb;
    
    vec3 color  = ((metal)? pow(clamp(dot(H,N), 0., 1.), shininess):clamp(dot(N,L),0.,1.))*baseColor;
    out_color = vec4(color,1.);
}
'''
)
queryProgramParameters(mainShaderProgram)

#https://moderngl.readthedocs.io/en/5.10.0/topics/buffer_format.html#syntax
teapot_renderable = gl.vertex_array(mainShaderProgram, [
    (teapot_vertex_buffer, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
]) #for vertex_buffer in vertex_buffers]

box_renderable = gl.vertex_array(mainShaderProgram, [
    (box_object_buffer, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
])

Main Program Parameters
in_position <class '_moderngl.Attribute'> <Attribute: 0>
in_normal <class '_moderngl.Attribute'> <Attribute: 1>
in_uv <class '_moderngl.Attribute'> <Attribute: 2>
model <class '_moderngl.Uniform'> <Uniform: 0>
view <class '_moderngl.Uniform'> <Uniform: 1>
perspective <class '_moderngl.Uniform'> <Uniform: 2>
eye <class '_moderngl.Uniform'> <Uniform: 3>
map <class '_moderngl.Uniform'> <Uniform: 4>
light <class '_moderngl.Uniform'> <Uniform: 5>
metal <class '_moderngl.Uniform'> <Uniform: 6>


#### Light Program and Light object Rendable

In [1604]:
lightShaderProgram = gl.program(
    vertex_shader=shared_vertex_shader_code,
    fragment_shader='''#version 330 core

in vec3 f_position;
in vec3 f_normal;

layout (location = 0) out vec4 out_color;

void main() {
    vec3 N = normalize(f_normal);
    out_color = vec4(0.5*(N+1.0),1.0);
}'''
)
print("Light Program Parameters")
queryProgramParameters(lightShaderProgram)

light_renderable = gl.vertex_array(lightShaderProgram, [
    (light_object_buffer, '3f 3f', 'in_position', 'in_normal')
])


shadowShaderProgram = gl.program(
    vertex_shader=shadow_vertex_shader_code,
    fragment_shader='''#version 330 core

layout (location = 0) out vec4 out_color;


void main() {
    out_color = vec4(0,0,0,0.5);
}'''
)


shadow_renderable = gl.vertex_array(shadowShaderProgram, [
    (Shadow_vertex_buffer, '3f', 'in_position')
])

Light Program Parameters
in_position <class '_moderngl.Attribute'> <Attribute: 0>
in_normal <class '_moderngl.Attribute'> <Attribute: 1>
model <class '_moderngl.Uniform'> <Uniform: 0>
view <class '_moderngl.Uniform'> <Uniform: 1>
perspective <class '_moderngl.Uniform'> <Uniform: 2>


#### Camera Parameters

In [1605]:
cameraDistance = 20
lookAtPoint = glm.vec3(0)
upVector = glm.vec3(0.0, 1.0, 0.0)
cameraStartDirection = glm.normalize(glm.vec3(0,10,20))
cameraOrbitAxis = glm.vec3(0,1,0)

def get_camera_matrix(angle):
    d = cameraStartDirection
    viewDirection = glm.rotate(glm.radians(angle), cameraOrbitAxis)*cameraStartDirection
    eyePoint = lookAtPoint + cameraDistance*viewDirection
    viewMatrix = glm.lookAt(eyePoint, lookAtPoint, upVector)
    return viewMatrix, eyePoint

near = 1
far = 45
fov = 60
def getPerspectiveMatrix(aspect_ratio):
    return glm.perspective(glm.radians(fov), aspect_ratio, near, far)

#### Light Object Parameters

In [1606]:
#start light vector 
lightDistance = 12 
lightStartDirection = glm.normalize(glm.vec3(1,1,0))
lightTarget = glm.vec3(0)
lightOrbitAxis = glm.vec3(0,1,0)
def getLightVector(angle):
    lightVector = glm.rotate(glm.radians(angle), lightOrbitAxis)*lightStartDirection
    lightPosition = lightTarget + lightDistance * lightVector
    return lightVector, lightPosition
    #return glm.normalize(glm.vec3(cos(angle_in_radian), 1, -sin(angle_in_radian)))# This will change for dynamic light

#### Method to render Light Object

In [1607]:
def drawLight(lightPosition, view, perspective):
    program = lightShaderProgram
    program['view'].write(view)
    program["perspective"].write(perspective)
    light_model_transformation = glm.translate(lightPosition)*light_object_scaletransformation
    program["model"].write(light_model_transformation)
    light_renderable.render()

#### Method to render Scene

In [1608]:
def drawScene(light, eye, view, perspective):
    program = mainShaderProgram
    program["eye"].write(eye)
    program["light"].write(light)
    program['view'].write(view)
    program["perspective"].write(perspective)
    
    program["metal"] = False  
    diffuse_sampler.use(0)
    program["map"]=0
    program["model"].write(box_object_transformation)
    box_renderable.render()
    
    program["metal"] = True
    specular_sampler.use(0)
    program["map"]=0
    program["model"].write(teapot_object_transformation)
    teapot_renderable.render()

#### Main Program

In [ ]:
# Toggles
pause = True
pointSourceFlag = False

angle = 0
lightVector = getLightVector(angle)
perspectiveMatrix = getPerspectiveMatrix(aspect_ratio)
viewMatrix, eyePoint = get_camera_matrix(0)

running = True
while running:   
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_l:
                pointSourceFlag = not pointSourceFlag
        elif (event.type == pygame.WINDOWRESIZED):
            aspect_ratio = event.x/event.y
            perspectiveMatrix = getPerspectiveMatrix(aspect_ratio)    
            
    gl.clear(0.1,0.1,0.1,depth=1.0)
    gl.enable(gl.DEPTH_TEST)

    lightVector, lightPosition = getLightVector(angle)
    
    drawLight(lightPosition, viewMatrix, perspectiveMatrix)

    light = glm.vec4(lightPosition,1) if pointSourceFlag else glm.vec4(lightVector,0)
    
    drawScene(light, eyePoint, viewMatrix, perspectiveMatrix)


    Q = glm.vec3(0,0.025, 0)  

    shadowShaderProgram['L'].write(light)
    shadowShaderProgram['Q'].write(Q)
    shadowShaderProgram['n'].write(glm.vec3(0,1,0))
    shadowShaderProgram['view'].write(viewMatrix)
    shadowShaderProgram['perspective'].write(perspectiveMatrix)
    shadowShaderProgram['model'].write(teapot_object_transformation)
    shadow_renderable.render()

    
    if (not pause):
        angle = angle+1
        if (angle > 360):
            angle = angle - 360

    pygame.display.flip()
pygame.quit()